## 빅데이터 실습

### 스타벅스 매장 입지 분석

#### 데이터 수집

##### 셀레니움 사용 자동화

In [25]:
# 필요 라이브러리 사용등록
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd

In [26]:
# 웹드라이버 스타벅스 웹사이트 오픈
driver = webdriver.Chrome()
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

In [27]:
# path = 'div.loca_step1_cint > ul > li:nth-child(1) > a'
# driver.find_element(By.CSS_SELECTOR, path).click()

In [31]:
## 웹드라이버에서 소스 위치의 서울 링크를 클릭
name = 'set_sido_cd_btn'
driver.find_elements(By.CLASS_NAME, name)[0].click()

In [ ]:
# ## 웹드라이버에서 소스 위치의 부산 링크를 클릭
# name = 'set_sido_cd_btn'
# driver.find_elements(By.CLASS_NAME, name)[5].click()

In [32]:
# 웹드라이버에서 소스 위치 전체 링크 클릭
name = 'set_gugun_cd_btn'
driver.find_elements(By.CLASS_NAME, name)[0].click()

##### BeautifulSoup로 html 가져오기

In [33]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


In [ ]:
sbSeoulList = soup.select('li.quickResultLstCon')
len(sbSeoulList)

In [34]:
sbSeoulList[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

##### 서울 매장 리스트 DF

In [36]:
sbSeoulStores = []
for item in sbSeoulList:
    name = item.select('strong')[0].text.strip() # 지점명
    lat = item['data-lat'].strip() # 위도
    lng = item['data-long'].strip() # 경도
    storeType = item.select('i')[0]['class'][0].split('_')[1] # 가게종류(일반, 리저브, DT, WT)
    address = str(item.select('p.result_details')[0]).split('<br/>')[0].split('>')[1] # 주소
    tel = str(item.select('p.result_details')[0]).split('<br/>')[1].split('<')[0] # 전화번호

    sbSeoulStores.append([name, lat, lng, storeType, address, tel])

In [ ]:
len(sbSeoulStores)

In [37]:
# DF로 변경
dfStarBucksSeoul = pd.DataFrame(sbSeoulStores, columns=['매장명', '위도', '경도', '매장 타입', '주소', '전화번호'])

In [38]:
dfStarBucksSeoul.tail()

,매장명,위도,경도,매장 타입,주소,전화번호
607,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310,1522-3232
608,상봉역,37.59689,127.08647,general,서울특별시 중랑구 망우로 307 (상봉동),1522-3232
609,묵동,37.615368,127.076633,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232
610,양원역,37.6066536267232,127.106359790053,general,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232
611,중화역,37.60170912407773,127.07841136432036,general,서울특별시 중랑구 봉화산로 35 1층,1522-3232


In [ ]:
# 결측치 확인
dfStarBucksSeoul.info()

In [ ]:
dfStarBucksSeoul.to_excel('./data/starbucks_seoyl_list.xlsx')

##### 서울 데이터 광장 OpenAPI 공공 데이터 수집

- 아래 방법 동작안함

In [ ]:
#서울 열린 데이터 광장 API 키
SEOUL_API_KEY = '465367456d76696e36364f445a7157'

In [ ]:
#서울 열린 데이터 광장 호출 URL
url = f'http://openapi.seoul.go.kr:8088/{SEOUL_API_KEY}/json/SdeTlSccoSigW/1/25'
url

In [ ]:
## 행정구역 데이터(위경도 + 구역)
- https://github.com/southkorea/seoul-maps